# EDA with Rust and Polars

## Setup deps

__Warning__: The next step may take some time on the first execution, have a look at the console, if the crates are being downloaded

In [5]:
:dep polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
:dep plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
:dep ndarray = {version = "0.15.6"}
:dep smartcore = {version = "0.3.2"}

In [6]:
:show_deps

ndarray = {version = "0.15.6"}
plotters = { version = "^0.3.5", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
polars = {version = "0.35.0", features = ["describe", "lazy", "ndarray"]}
smartcore = {version = "0.3.2"}


## Import modules

In [7]:
use polars::prelude::*;
use polars::frame::DataFrame;
use std::path::Path;
use ndarray::{ArrayBase, DataMut, Dimension, concatenate, Axis};
use plotters::prelude::*;
use smartcore::linalg::basic::matrix::DenseMatrix;
use ndarray::prelude::*;

### Docs

You can display the `man` page of a crate by running `:doc DataFrame`

In [9]:
// :doc DataFrame

## Load data from csv

*Example taken from [Rust-Data-Analysis/1-Iris Dataset](https://github.com/wiseaidev/rust-data-analysis/blob/main/1-iris-data-analysis-rust.ipynb)*

In [10]:
fn read_data_frame_from_csv(
    csv_file_path: &Path,
) -> DataFrame {
    CsvReader::from_path(csv_file_path)
        .expect("Cannot open file.")
        .has_header(true)
        .finish()
        .unwrap()
}

let iris_file_path: &Path = Path::new("dataset/Iris.csv");
let iris_df: DataFrame = read_data_frame_from_csv(iris_file_path);
iris_df

shape: (150, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa    │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa    │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa    │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …              │
│ 147 ┆ 6.3           ┆ 2.5          ┆ 5.0           ┆ 1.9          ┆ Iris-virginica │
│ 148 ┆ 6.5           ┆ 3.0

## EDA - Part One, explore the Iris Dataset

In [11]:
// Shape
iris_df.shape()

(150, 6)

In [13]:
// Show first rows
iris_df.head(Some(10))

shape: (10, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species     │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---         │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str         │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪═════════════╡
│ 1   ┆ 5.1           ┆ 3.5          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 2   ┆ 4.9           ┆ 3.0          ┆ 1.4           ┆ 0.2          ┆ Iris-setosa │
│ 3   ┆ 4.7           ┆ 3.2          ┆ 1.3           ┆ 0.2          ┆ Iris-setosa │
│ 4   ┆ 4.6           ┆ 3.1          ┆ 1.5           ┆ 0.2          ┆ Iris-setosa │
│ …   ┆ …             ┆ …            ┆ …             ┆ …            ┆ …           │
│ 7   ┆ 4.6           ┆ 3.4          ┆ 1.4           ┆ 0.3          ┆ Iris-setosa │
│ 8   ┆ 5.0           ┆ 3.4          ┆ 1.5           ┆ 0.2   

In [15]:
// Describe
iris_df.describe(None)?

shape: (9, 7)
┌────────────┬───────────┬──────────────┬──────────────┬──────────────┬──────────────┬─────────────┐
│ describe   ┆ Id        ┆ SepalLengthC ┆ SepalWidthCm ┆ PetalLengthC ┆ PetalWidthCm ┆ Species     │
│ ---        ┆ ---       ┆ m            ┆ ---          ┆ m            ┆ ---          ┆ ---         │
│ str        ┆ f64       ┆ ---          ┆ f64          ┆ ---          ┆ f64          ┆ str         │
│            ┆           ┆ f64          ┆              ┆ f64          ┆              ┆             │
╞════════════╪═══════════╪══════════════╪══════════════╪══════════════╪══════════════╪═════════════╡
│ count      ┆ 150.0     ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150.0        ┆ 150         │
│ null_count ┆ 0.0       ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0.0          ┆ 0           │
│ mean       ┆ 75.5      ┆ 5.843333     ┆ 3.054        ┆ 3.758667     ┆ 1.198667     ┆ null        │
│ std        ┆ 43.445368 ┆ 0.828066     ┆ 0.433594     ┆ 1.76442      ┆ 0.763

In [17]:
// Iterate through columns
let column_names = iris_df.get_column_names(); 
{
column_names
}

["Id", "SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm", "Species"]

### Mean

In [18]:
iris_df.mean()

shape: (1, 6)
┌──────┬───────────────┬──────────────┬───────────────┬──────────────┬─────────┐
│ Id   ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species │
│ ---  ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---     │
│ f64  ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str     │
╞══════╪═══════════════╪══════════════╪═══════════════╪══════════════╪═════════╡
│ 75.5 ┆ 5.843333      ┆ 3.054        ┆ 3.758667      ┆ 1.198667     ┆ null    │
└──────┴───────────────┴──────────────┴───────────────┴──────────────┴─────────┘

## Max

In [19]:
iris_df.max()

shape: (1, 6)
┌─────┬───────────────┬──────────────┬───────────────┬──────────────┬────────────────┐
│ Id  ┆ SepalLengthCm ┆ SepalWidthCm ┆ PetalLengthCm ┆ PetalWidthCm ┆ Species        │
│ --- ┆ ---           ┆ ---          ┆ ---           ┆ ---          ┆ ---            │
│ i64 ┆ f64           ┆ f64          ┆ f64           ┆ f64          ┆ str            │
╞═════╪═══════════════╪══════════════╪═══════════════╪══════════════╪════════════════╡
│ 150 ┆ 7.9           ┆ 4.4          ┆ 6.9           ┆ 2.5          ┆ Iris-virginica │
└─────┴───────────────┴──────────────┴───────────────┴──────────────┴────────────────┘

## Conversion to multidimensional array

In [20]:
// Drop "Species" Categorical column, since we are going to work with numerical values from now on
let numeric_iris_df: DataFrame = iris_df.drop("Species")?;

let numeric_iris_ndarray: ArrayBase<_, _> = numeric_iris_df.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap();
numeric_iris_ndarray

[[1.0, 5.1, 3.5, 1.4, 0.2],
 [2.0, 4.9, 3.0, 1.4, 0.2],
 [3.0, 4.7, 3.2, 1.3, 0.2],
 [4.0, 4.6, 3.1, 1.5, 0.2],
 [5.0, 5.0, 3.6, 1.4, 0.2],
 ...,
 [146.0, 6.7, 3.0, 5.2, 2.3],
 [147.0, 6.3, 2.5, 5.0, 1.9],
 [148.0, 6.5, 3.0, 5.2, 2.0],
 [149.0, 6.2, 3.4, 5.4, 2.3],
 [150.0, 5.9, 3.0, 5.1, 1.8]], shape=[150, 5], strides=[1, 150], layout=Ff (0xa), const ndim=2

## Show first element

In [21]:
numeric_iris_ndarray[[0, 0]]

1.0

### First row

In [22]:
numeric_iris_ndarray.slice(s![0, ..])

[1.0, 5.1, 3.5, 1.4, 0.2], shape=[5], strides=[150], layout=Custom (0x0), const ndim=1

### First column

In [24]:
// First columne
numeric_iris_ndarray.column(1)

[5.1, 4.9, 4.7, 4.6, 5.0, 5.4, 4.6, 5.0, 4.4, 4.9, 5.4, 4.8, 4.8, 4.3, 5.8, 5.7, 5.4, 5.1, 5.7, 5.1, 5.4, 5.1, 4.6, 5.1, 4.8, 5.0, 5.0, 5.2, 5.2, 4.7, 4.8, 5.4, 5.2, 5.5, 4.9, 5.0, 5.5, 4.9, 4.4, 5.1, 5.0, 4.5, 4.4, 5.0, 5.1, 4.8, 5.1, 4.6, 5.3, 5.0, 7.0, 6.4, 6.9, 5.5, 6.5, 5.7, 6.3, 4.9, 6.6, 5.2, 5.0, 5.9, 6.0, 6.1, 5.6, 6.7, 5.6, 5.8, 6.2, 5.6, 5.9, 6.1, 6.3, 6.1, 6.4, 6.6, 6.8, 6.7, 6.0, 5.7, 5.5, 5.5, 5.8, 6.0, 5.4, 6.0, 6.7, 6.3, 5.6, 5.5, 5.5, 6.1, 5.8, 5.0, 5.6, 5.7, 5.7, 6.2, 5.1, 5.7, 6.3, 5.8, 7.1, 6.3, 6.5, 7.6, 4.9, 7.3, 6.7, 7.2, 6.5, 6.4, 6.8, 5.7, 5.8, 6.4, 6.5, 7.7, 7.7, 6.0, 6.9, 5.6, 7.7, 6.3, 6.7, 7.2, 6.2, 6.1, 6.4, 7.2, 7.4, 7.9, 6.4, 6.3, 6.1, 7.7, 6.3, 6.4, 6.0, 6.9, 6.7, 6.9, 5.8, 6.8, 6.7, 6.7, 6.3, 6.5, 6.2, 5.9], shape=[150], strides=[1], layout=CFcf (0xf), const ndim=1

## Data Visualization

In [25]:
let sepal_samples:Vec<(f64,f64)> = {
    let sepal_length_cm: DataFrame = iris_df.select(vec!["SepalLengthCm"]).unwrap();
    let mut sepal_length = sepal_length_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    let sepal_width_cm: DataFrame = iris_df.select(vec!["SepalWidthCm"]).unwrap();
    let mut sepal_width = sepal_width_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    sepal_width.zip(sepal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(1f64..5f64, 3f64..9f64)?;
    
    chart.configure_mesh()
        .x_desc("Sepal Length (cm)")
        .y_desc("Sepal Width (cm)")
        .draw()?;
    
    chart.draw_series(sepal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, BLUE.filled())));

    Ok(())
}).style("width:60%")


Iris Dataset
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Sepal Width (cm)
 
 
Sepal Length (cm)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
9.0
 
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0
 
 
 
3.5
 
 
 
4.0
 
 
 
4.5
 
 
 
5.0

## Petal

In [28]:
let petal_samples: Vec<(f64,f64)> = {
    let petal_length_cm: DataFrame = iris_df.select(vec!["PetalLengthCm"]).unwrap();
    let mut petal_length = petal_length_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    let petal_width_cm: DataFrame = iris_df.select(vec!["PetalWidthCm"]).unwrap();
    let mut petal_width = petal_width_cm.to_ndarray::<Float64Type>(IndexOrder::Fortran).unwrap().into_raw_vec().into_iter();
    petal_width.zip(petal_length).collect()
};

evcxr_figure((640, 480), |root| {
    let mut chart = ChartBuilder::on(&root)
        .caption("Iris Dataset", ("Arial", 30).into_font())
        .x_label_area_size(40)
        .y_label_area_size(40)
        .build_cartesian_2d(0f64..3f64, 0f64..8f64)?;
    
    chart.configure_mesh()
        .x_desc("Petal Length (cm)")
        .y_desc("Petal Width (cm)")
        .draw()?;
    
    chart.draw_series(petal_samples.iter().map(|(x, y)| Circle::new((*x,*y), 3, BLUE.filled())));

    Ok(())
}).style("width:60%")


Iris Dataset
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
Petal Width (cm)
 
 
Petal Length (cm)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
0.0
 
 
 
1.0
 
 
 
2.0
 
 
 
3.0
 
 
 
4.0
 
 
 
5.0
 
 
 
6.0
 
 
 
7.0
 
 
 
8.0
 
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
3.0

## Histogram

In [30]:
evcxr_figure((640, 480), |root| {
    _ = root.fill(&WHITE);

    let mut chart = ChartBuilder::on(&root)
        .caption("Histogram", ("Arial", 20).into_font())
        .x_label_area_size(50)
        .y_label_area_size(50)
        .build_cartesian_2d(1u32..5u32, 0f64..1f64)?;

    chart.configure_mesh()
        .disable_x_mesh()
        .disable_y_mesh()
        .y_labels(5)
        .y_label_formatter(&|y| format!("{}%", (*y * 100.0) as u32))
        .draw()?;

    let hist = Histogram::vertical(&chart)
        .style(GREEN.filled())
        .margin(0)
        .data(sepal_samples.iter().map(|(x,_)| (*x as u32, 0.01)));

    let _ = chart.draw_series(hist);
    
    Ok(())
}).style("width:100%")

Histogram
 
 
 
0%
 
 
 
50%
 
 
 
100%
 
 
 
 
1
 
 
 
2
 
 
 
3
 
 
 
4
 
 
 
5